# data upload

In [1]:
import os
base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
os.chdir(base)

In [2]:
from sqlalchemy import create_engine, types
from datetime import datetime, timedelta
from IPython.display import display
import matplotlib.pyplot as plt
import reliant_scrape as rs
import pyodbc
import pandas as pd
import numpy as np
import pprint
import yaml
import json
import sys
import os

base = os.path.join('C:\\', 'Users', 'steve', 'Documents', 'reliant-scrape')
data = os.path.join(base, 'data')
pp = pprint.PrettyPrinter(indent = 1)

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader = yaml.FullLoader)
        
with open('db_creds.json', 'r') as f:
    db_creds = json.load(f)
    
print('loaded credentials & imported modules.')

loaded credentials & imported modules.


In [3]:
os.chdir(data)
stage = pd.read_csv('master_usage_03132021_missing.csv')
stage['Date'] = pd.to_datetime(stage['Date'])

In [4]:
stage

,Date,Usage (kWh),Cost ($),Hi,Low
0,2021-03-07 00:00:00,0.8,0.10,68,45
1,2021-03-07 01:00:00,0.4,0.06,68,45
2,2021-03-07 02:00:00,0.2,0.03,68,45
3,2021-03-07 03:00:00,0.2,0.03,68,45
4,2021-03-07 04:00:00,0.2,0.03,68,45
...,...,...,...,...,...
307,2021-02-12 19:00:00,1.7,0.22,40,37
308,2021-02-12 20:00:00,1.2,0.15,40,37
309,2021-02-12 21:00:00,0.4,0.06,40,37
310,2021-02-12 22:00:00,0.8,0.11,40,37


In [5]:
rs.table_upload(stage, 'reliant_energy_db', 'daily_use', db_creds)
print('uploaded data.')

wrote df to sql table.
uploaded data.
